## Initialize Terraform

In [1]:
#rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## View `variables.tf`

- Note that we have added a variable for the `kubernetes_version`.

In [2]:
#!type variables.tf #use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.
# Note:
# - The block type is "variable" since we are defining a Terraform variable.
# - "app_name", "location" and "kubernetes_version" is the names of the Terraform variables.
# - "default" is used to set the value for a Terraform variable.
# - if "default" is omitted, Terraform will ask the user to input the value for the variable during "terraform apply".

variable "app_name" {
  default = "tsfn14g00"
}

variable "location" {
  default = "westeurope"
}

variable "kubernetes_version" {
  default = "1.27.7"
}


## View `resource-group.tf`

In [3]:
#!type resource-group.tf #sue this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}


## View `container-registry.tf`

In [4]:
!type container-registry.tf #use this on Windows
!cat container-registry.tf

# Creates a container registry in Azure (for Docker images).
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name" and "location" below are set from Terraform variables defined in the file "variables.tf".


resource "azurerm_container_registry" "main" {
  name                = var.app_name
  resource_group_name = azurerm_resource_group.main.name
  location            = var.location
  admin_enabled       = true
  sku                 = "Basic"
}


## View `kubernetes-cluster.tf`

In [5]:
#!type kubernetes-cluster.tf #use this on Windows
!cat kubernetes-cluster.tf

# Creates a managed Kubernetes cluster on Azure.
# Note!
# - Resource "azurerm_resource_group.main" with a property "name" is defined in the file "resource-group.tf".
# - The value for "resource_group_name" below is set using property "name" in resource "azurerm_resource_group.main":
#   - resource_group_name = azurerm_resource_group.main.name
# - "name", "location" and "kubernetes_version" below are set from Terraform variables defined in the file "variables.tf".
# Note 2!
# - We also create a "networkwatcher" (in its own Resource Group).
#   - This is required when a virtual network is created in Azure.
#   - This is automatically created by Azure, but we explicitly create it here.
#     - The only reason we do this explicitly is so Terraform Destroy will automatically delete it for us.

resource "azurerm_resource_group" "networkwatcher" {
  name     = "NetworkWatcherRG"
  location = var.location
}

resource "azurerm_network_watcher" "networkwatcher" {
  name                = "Networ

## Terraform Apply

In [6]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_container_registry.main will be created
  + resource "azurerm_container_registry" "main" {
      + admin_enabled                 = true
      + admin_password                = (sensitive value)
      + admin_username                = (known after apply)
      + encryption                    = (known after apply)
      + export_policy_enabled         = true
      + id                            = (known after apply)
      + location                      = "westeurope"
      + login_server                  = (known after apply)
      + name                          = "tsfn14g00"
      + network_rule_bypass_option    = "AzureServices"
      + network_rule_set              = (known after apply)
      + public_network_access_enabled = true
      + resource_group_name           = "tsf

## List Azure Resource Groups

In [7]:
!az group list -o table

Name                               Location    Status
---------------------------------  ----------  ---------
tsfn14g00                          westeurope  Succeeded
NetworkWatcherRG                   westeurope  Succeeded
MC_tsfn14g00_tsfn14g00_westeurope  westeurope  Succeeded


## List Azure Container Registries

In [8]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-02T20:30:14Z  True


## Build and Push a Docker Image to the Azure Container Registry

- Note, this code has only been tested on Ubuntu.

In [9]:
# Get connection information to the Azure Container Registry
export CONTAINER_REGISTRY_LOGIN_SERVER=$(az acr show -n tsfn14g00 --query loginServer | sed 's/^"//;s/"$//')
export CONTAINER_REGISTRY_USERNAME=$(az acr credential show -n tsfn14g00 --query username | sed 's/^"//;s/"$//')
export CONTAINER_REGISTRY_PASSWORD=$(az acr credential show -n tsfn14g00 --query passwords[0].value | sed 's/^"//;s/"$//')
echo -e "\nCONTAINER_REGISTRY_LOGIN_SERVER: $CONTAINER_REGISTRY_LOGIN_SERVER"
echo -e "CONTAINER_REGISTRY_USERNAME: $CONTAINER_REGISTRY_USERNAME"
echo -e "CONTAINER_REGISTRY_PASSWORD: $CONTAINER_REGISTRY_PASSWORD"

# Login to the Azure COntainer Registry via Docker
echo $CONTAINER_REGISTRY_PASSWORD | docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME --password-stdin  > /dev/null 2>&1

# Prepare Nodejs Application
#rm -rf ./video-streaming/node_modules ./video-streaming/package.json ./video-streaming/package-lock.json
cd ./video-streaming && \
npm init -y && \
npm pkg set 'main'='./src/index.js' && \
npm pkg set 'scripts.start'='node ./src/index.js' && \
npm pkg set 'scripts.start:dev'='nodemon ./src/index.js' && \
npm pkg delete 'scripts.test' && \
npm install --silent --save express@5.0.0-beta.1 && \
npm install --silent --save-dev nodemon@3.0.3 && \
cd ..

# Build Docker image with Nodejs Application
docker build -q -t $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1 -f ./video-streaming/Dockerfile ./video-streaming

# Push Docker Image to Azure Container Registry
docker push $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1

# Clean up
docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/video-streaming:1 > /dev/null 2>&1
docker image list | grep -i -E 'REPOSITORY|video-streaming'

# Logout from the Azure COntainer Registry via Docker
docker logout $CONTAINER_REGISTRY_LOGIN_SERVER > /dev/null 2>&1

# List Images in the Azure Container Registry
az acr repository list -n tsfn14g00 --top 10 -o table


CONTAINER_REGISTRY_LOGIN_SERVER: tsfn14g00.azurecr.io
CONTAINER_REGISTRY_USERNAME: tsfn14g00
CONTAINER_REGISTRY_PASSWORD: Kk/oMsn+o932FSJ1XGa+Y90u3Tm5IGyREdYh/Nlij5+ACRA3fgko
Wrote to /home/patrick/projects/tsfn14/04_Kubernetes_Cluster/video-streaming/package.json:

{
  "name": "video-streaming",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}


(##################) ⠇ reify:nodemon: timing reifyNode:node_modules/semver Compm Comp[?25h(⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠦ idealTree:video-streaming: sill idealTree buildDeps
failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

sha256:dcc6cbb0f2c

## List Azure Kubernetes Services

In [10]:
!az aks list -o table

Name       Location    ResourceGroup    KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------  ----------  ---------------  -------------------  --------------------------  -------------------  -------------------------------------------
tsfn14g00  westeurope  tsfn14g00        1.27.7               1.27.7                      Succeeded            tsfn14g00-e5hcd6rd.hcp.westeurope.azmk8s.io


## Add Azure Kubernetes Cluster Info. to Local Kubectl Config File

- `~/.kube/config`

In [11]:
!az aks get-credentials --name tsfn14g00 --resource-group tsfn14g00

# The command below attaches the Azure Kubernetes Cluster to the Azure Container Registry so that
# the Azure Kubernetes Cluster can pull images from the Azure Container Registry, but we don't
# have to do this explicitly here, since we do it in the Terraform file "kubernetes-cluster.tf".
#az aks update -n tsfn14g00 -g tsfn14g00 --attach-acr tsfn14g00 -o table

Merged "tsfn14g00" as current context in /home/patrick/.kube/config


## Ensure the Kubectl Context is set to the Azure Kubernetes Cluster

In [12]:
!kubectl config get-contexts
#!kubectl config use-context tsfn14g00
!kubectl config current-context
#!kubectl config view

CURRENT   NAME                          CLUSTER      AUTHINFO                          NAMESPACE
          kubernetes-admin@kubernetes   kubernetes   kubernetes-admin                  
          minikube                      minikube     minikube                          default
*         tsfn14g00                     tsfn14g00    clusterUser_tsfn14g00_tsfn14g00   
tsfn14g00


## List Deployments, Pods and Services

- Notice we have a ClusterIP Service to the Azure Kubernetes Cluster already defined before deploying any resources ourselves.

In [13]:
!kubectl get deployment
!kubectl get pods
!kubectl get services

No resources found in default namespace.
No resources found in default namespace.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   4m22s


## Apply Deployment to the Azure Kubernets Cluster

In [14]:
!kubectl apply -f manifests/deployment.yaml

deployment.apps/video-streaming created
service/video-streaming created


## List Deployments, Pods and Services

- Note that we store the Load Balancer's Public IP in an environment variable so we can use it in the notebook
  - `LOADBALANCER_PUBLIC_IP`
- Note, this code has only been tested on Ubuntu.

In [15]:
!kubectl get deployment
!kubectl get pods
!kubectl get services

!export LOADBALANCER_PUBLIC_IP=$(kubectl get service video-streaming -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
!echo -e "\nLOADBALANCER_PUBLIC_IP: $LOADBALANCER_PUBLIC_IP"

NAME              READY   UP-TO-DATE   AVAILABLE   AGE
video-streaming   1/1     1            1           21s
NAME                               READY   STATUS    RESTARTS   AGE
video-streaming-864b6976bf-gptlh   1/1     Running   0          21s
NAME              TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)        AGE
kubernetes        ClusterIP      10.0.0.1     <none>        443/TCP        6m27s
video-streaming   LoadBalancer   10.0.18.62   20.86.199.6   80:30001/TCP   22s

LOADBALANCER_PUBLIC_IP: 20.86.199.6


## Test the Application in the Kubernetes Cluster

In [16]:
!firefox http://$LOADBALANCER_PUBLIC_IP/video

[GFX1-]: glxtest: ManageChildProcess failed



## Delete Deployment in the Azure Kubernets Cluster

In [17]:
!kubectl delete -f manifests/deployment.yaml

deployment.apps "video-streaming" deleted
service "video-streaming" deleted


## List Deployments, Pods and Services

In [18]:
!kubectl get deployment
!kubectl get pods
!kubectl get services

No resources found in default namespace.
No resources found in default namespace.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   7m34s


## Ensure the Kubectl Context is set to the Minikube Cluster

In [19]:
!kubectl config use-context minikube
!kubectl config current-context

Switched to context "minikube".
minikube


## Remove Azure Kubernetes Cluster Info. from Local Kubectl Config File

- `~/.kube/config`

In [20]:
!kubectl config delete-cluster tsfn14g00
!kubectl config delete-context tsfn14g00
!kubectl config delete-user clusterUser_tsfn14g00_tsfn14g00
#!kubectl config view

deleted cluster tsfn14g00 from /home/patrick/.kube/config
deleted context tsfn14g00 from /home/patrick/.kube/config
deleted user clusterUser_tsfn14g00_tsfn14g00 from /home/patrick/.kube/config


## Terraform Destroy

In [21]:
!terraform destroy -auto-approve

azurerm_resource_group.networkwatcher: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/NetworkWatcherRG]
azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_network_watcher.networkwatcher: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/NetworkWatcherRG/providers/Microsoft.Network/networkWatchers/NetworkWatcher_westeurope]
azurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]
azurerm_kubernetes_cluster.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerService/managedClusters/tsfn14g00]
azurerm_role_assignment.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/reso

## List Azure Kubernetes Services

In [22]:
!az aks list -o table

## List Azure Container Registries

In [23]:
!az acr list -o table

## List Azure Resource Groups

In [24]:
!az group list -o table